# HW2 - Pandas and ICD-codes

### Get the data

For this assignment, we'll need to get some data! We will be using the Diabetes Dataset that is located here:

https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008#

Afterwards, unzip the folder and place the contents in a folder called `/data/` at this directory.

Make sure to add a file called `.gitignore` at the root of your directory and add the line 

```
    data/
```
to it so that it ignore any files that you place in the Data folder.

In [1]:
import pandas as pd
import numpy as np
import string
import re
import math

In [2]:
diabetes_df = pd.read_csv('diabetic_data.csv')
diabetes_df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


![](data/datadictionary.png)

## Data Source [5 pts]

When we begin working with data, it is important to try and understand the data that we've been given. Often the context can tell us a lot of information about the data. In fact, often times understanding what is *not* in the data is just as critical as understanding the data itself. From the above link, the information about how the data was collected can be found in this [paper](https://www.hindawi.com/journals/bmri/2014/781670/)

List the 5 inclusion criteria to be in the dataset: 

1. It is an inpatient encounter (a hospital admission).
2. It is a “diabetic” encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.
3. The length of stay was at least 1 day and at most 14 days.
4. Laboratory tests were performed during the encounter.
5. Medications were administered during the encounter.


## Explore the Data [55 pts]
Using the data, answer the following questions:

#### 1. How many rows does the `diabetes_df` have? How many columns? [2.5 pts]

In [3]:
print(diabetes_df.shape)

(101766, 50)


```diabetes_df``` has 101766 rows, and 50 columns.

#### 2. How many unique encounters are there? How many unique patients? [2.5 pts]

In [4]:
e = diabetes_df['encounter_id'].unique()
p = diabetes_df['patient_nbr'].unique()
print('There are {} unique encounters and {} unique patients.'.format(len(e),len(p)))

There are 101766 unique encounters and 71518 unique patients.


#### 3. What is the most amount of encounters that a single patient has in the dataset? [5 pts]

In [5]:
diabetes_df['patient_nbr'].value_counts()

88785891     40
43140906     28
23199021     23
1660293      23
88227540     23
23643405     22
84428613     22
92709351     21
23398488     20
90609804     20
88789707     20
37096866     20
89472402     20
29903877     20
88681950     19
88479036     19
97391007     19
24011577     18
3481272      18
91160280     18
84348792     18
3401055      18
91751121     18
106757478    17
90489195     17
41699412     17
84676248     16
384939       16
90164655     16
41617368     16
             ..
141459593     1
54207855      1
71579169      1
23406147      1
6348348       1
137952824     1
23234103      1
78943797      1
43683723      1
85241394      1
18267696      1
45161577      1
32417442      1
61105707      1
106231896     1
3397149       1
39734766      1
23850522      1
42977016      1
113160366     1
8105490       1
16600590      1
92990970      1
783198        1
105551478     1
71081460      1
30060018      1
67443444      1
141344240     1
93251151      1
Name: patient_nbr, Lengt

In [6]:
df = diabetes_df['patient_nbr'].value_counts().tolist()

print('The most amount of encounters that a single patient has in the dataset is {}.'.format(df[0]))

The most amount of encounters that a single patient has in the dataset is 40.


#### 4. Show the proportion of non-missing values are in the dataset for each column. [10 pts]
> Make sure you check to see if there are missing values that aren't coded as missing, but should be

In [7]:
row = diabetes_df.shape[0]
for col in diabetes_df.columns:
    collist = diabetes_df[col].tolist()
    prop = 1 - ( collist.count('?') + collist.count('Unknown/Invalid') + collist.count('None'))/row
    print('{} : {:.1%}'.format(col,prop))

encounter_id : 100.0%
patient_nbr : 100.0%
race : 97.8%
gender : 100.0%
age : 100.0%
weight : 3.1%
admission_type_id : 100.0%
discharge_disposition_id : 100.0%
admission_source_id : 100.0%
time_in_hospital : 100.0%
payer_code : 60.4%
medical_specialty : 50.9%
num_lab_procedures : 100.0%
num_procedures : 100.0%
num_medications : 100.0%
number_outpatient : 100.0%
number_emergency : 100.0%
number_inpatient : 100.0%
diag_1 : 100.0%
diag_2 : 99.6%
diag_3 : 98.6%
number_diagnoses : 100.0%
max_glu_serum : 5.3%
A1Cresult : 16.7%
metformin : 100.0%
repaglinide : 100.0%
nateglinide : 100.0%
chlorpropamide : 100.0%
glimepiride : 100.0%
acetohexamide : 100.0%
glipizide : 100.0%
glyburide : 100.0%
tolbutamide : 100.0%
pioglitazone : 100.0%
rosiglitazone : 100.0%
acarbose : 100.0%
miglitol : 100.0%
troglitazone : 100.0%
tolazamide : 100.0%
examide : 100.0%
citoglipton : 100.0%
insulin : 100.0%
glyburide-metformin : 100.0%
glipizide-metformin : 100.0%
glimepiride-pioglitazone : 100.0%
metformin-rosig

#### 5. For all numeric columns, show summary statistics (mean, median, max, min, etc) [2.5 pts]

In [8]:
import copy
diabetes_df.replace({'?':np.nan,'Unknown/Invalid':np.nan,'None':np.nan}, inplace = True)
diabetes_df = diabetes_df.apply(pd.to_numeric, errors = 'ignore')
diabetes_df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


#### 6. For all columns with a `dtype` of object, show the count of all of the values in that column [5 pts]
> You may want to reference the `pd.DataFrame.select_dtypes()` function 

In [9]:
df = diabetes_df.select_dtypes(include = 'object')
{col: df[col].value_counts() for col in df}

{'race': Caucasian          76099
 AfricanAmerican    19210
 Hispanic            2037
 Other               1506
 Asian                641
 Name: race, dtype: int64, 'gender': Female    54708
 Male      47055
 Name: gender, dtype: int64, 'age': [70-80)     26068
 [60-70)     22483
 [50-60)     17256
 [80-90)     17197
 [40-50)      9685
 [30-40)      3775
 [90-100)     2793
 [20-30)      1657
 [10-20)       691
 [0-10)        161
 Name: age, dtype: int64, 'weight': [75-100)     1336
 [50-75)       897
 [100-125)     625
 [125-150)     145
 [25-50)        97
 [0-25)         48
 [150-175)      35
 [175-200)      11
 >200            3
 Name: weight, dtype: int64, 'payer_code': MC    32439
 HM     6274
 SP     5007
 BC     4655
 MD     3532
 CP     2533
 UN     2448
 CM     1937
 OG     1033
 PO      592
 DM      549
 CH      146
 WC      135
 OT       95
 MP       79
 SI       55
 FR        1
 Name: payer_code, dtype: int64, 'medical_specialty': InternalMedicine                     14635
 

#### 7. What is the average number of labs administered by age category [2.5 pts]

In [10]:
diabetes_df.groupby(['age'])['num_lab_procedures'].mean()

age
[0-10)      41.012422
[10-20)     43.096961
[20-30)     43.066385
[30-40)     43.033642
[40-50)     42.785958
[50-60)     42.611961
[60-70)     42.600632
[70-80)     43.157396
[80-90)     44.085015
[90-100)    44.695310
Name: num_lab_procedures, dtype: float64

#### 8. Does the number of diagnoses equal the number of non-NA entries in the diag_* columns? [2.5 pts]

In [11]:
((~diabetes_df.filter(regex = 'diag_.*').isnull()).sum(axis = 1)) == diabetes_df['number_diagnoses']


0          True
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17         True
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25         True
26        False
27         True
28        False
29        False
          ...  
101736    False
101737    False
101738    False
101739    False
101740    False
101741    False
101742    False
101743    False
101744    False
101745     True
101746    False
101747    False
101748    False
101749    False
101750    False
101751    False
101752    False
101753    False
101754    False
101755    False
101756    False
101757    False
101758    False
101759    False
101760    False
101761    False
101762    False
101763    False
101764    False
101765    False
Length: 101766, dtype: b

#### 9. Create a new column that has the value of 1 if the medical specialty in that row contains the word Surgery and 0 otherwise. [10 pts]

In [12]:

diabetes_df['new column'] = pd.Series(index=diabetes_df.index)
diabetes_df['new column'] = [1 if  str(cell).find('Surgery')!=-1  else 0 for cell in diabetes_df['medical_specialty']]
diabetes_df['new column']

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        1
28        0
29        0
         ..
101736    0
101737    0
101738    0
101739    0
101740    0
101741    0
101742    0
101743    0
101744    0
101745    0
101746    0
101747    0
101748    0
101749    0
101750    0
101751    0
101752    0
101753    0
101754    0
101755    0
101756    0
101757    0
101758    0
101759    0
101760    0
101761    0
101762    0
101763    0
101764    1
101765    0
Name: new column, Length: 101766, dtype: int64

#### 10. How many encounters where the patient was between the ages of 0 and 20 took place with Pediatric providers? [5 pts]

In [13]:
diabetes_df['medical_specialty'] = [1 if  str(cell).find('Pediatric')!=-1 else 0 for cell in diabetes_df['medical_specialty']]
diabetes_df.groupby(['age','medical_specialty']).size()

age       medical_specialty
[0-10)    0                       32
          1                      129
[10-20)   0                      391
          1                      300
[20-30)   0                     1652
          1                        5
[30-40)   0                     3767
          1                        8
[40-50)   0                     9649
          1                       36
[50-60)   0                    17218
          1                       38
[60-70)   0                    22458
          1                       25
[70-80)   0                    26041
          1                       27
[80-90)   0                    17186
          1                       11
[90-100)  0                     2792
          1                        1
dtype: int64

In [14]:
print('There are {} encounters where the patient was between the ages of 0 and 20 took place with Pediatric providers.'.format(429))

There are 429 encounters where the patient was between the ages of 0 and 20 took place with Pediatric providers.


#### 12. find the counts of each of the available A1Cresult categories, broken down by whether or not the patient was readmitted (regardless of whether it was less than or greater than 30 days) [2.5 pts]

In [15]:

diabetes_df['readmitted'] = diabetes_df['readmitted'].apply({'>30':'Yes', '<30':'Yes','NO':'NO'}.get)
diabetes_df.groupby(['A1Cresult','readmitted']).size()

A1Cresult  readmitted
>7         NO            2129
           Yes           1683
>8         NO            4504
           Yes           3712
Norm       NO            2909
           Yes           2081
dtype: int64


## Single-level CCS categories [40 pts]

The columns `diag_1`, `diag_2`,  and `diag_3` contain ICD-9-CM codes for the encounters that took place in this dataset. However, if we count up the number of unique values between the 3 columns, we can see that the data is very sparse. 

As we discussed in class, the single-level CCS categories can be used instead to group similar ICD codes together. Download the latest version of the ICD-9-CM single-level CCS here: [https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Single_Level_CCS_2015.zip](https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Single_Level_CCS_2015.zip)

Unzip this and put the contents in the `data` folder along with the Diabetes dataset and read the file called `$dxref 2015.csv` into a variable. **NOTE** You must skip the first row of this `csv` file when reading it in because there is a note there that is unrelated to the contents of the data. Look up how to do this using the `pd.read_csv` function

In [16]:
dxref_df = pd.read_csv('$dxref 2015.csv',skiprows=[0])
dxref_df.columns

Index([''ICD-9-CM CODE'', ''CCS CATEGORY'', ''CCS CATEGORY DESCRIPTION'',
       ''ICD-9-CM CODE DESCRIPTION'', ''OPTIONAL CCS CATEGORY'',
       ''OPTIONAL CCS CATEGORY DESCRIPTION''],
      dtype='object')

Examine the contents of the data. Please bear in mind that Jupyter notebooks do not render whitespace or quotes very well sometimes, so watch out for that. Make sure you examine column names with `.columns` instead of just calling `.head()` and visually inspecting, for example.

In [17]:
dxref_df.head()

,'ICD-9-CM CODE','CCS CATEGORY','CCS CATEGORY DESCRIPTION','ICD-9-CM CODE DESCRIPTION','OPTIONAL CCS CATEGORY','OPTIONAL CCS CATEGORY DESCRIPTION'
0,' ','0 ','No DX',INVALID CODES IN USER DATA,' ',' '
1,'01000','1 ','Tuberculosis',PRIM TB COMPLEX-UNSPEC,' ',' '
2,'01001','1 ','Tuberculosis',PRIM TB COMPLEX-NO EXAM,' ',' '
3,'01002','1 ','Tuberculosis',PRIM TB COMPLEX-EXM UNKN,' ',' '
4,'01003','1 ','Tuberculosis',PRIM TB COMPLEX-MICRO DX,' ',' '


If you notice, the ICD codes are not very well-formatted in either the Diabetes dataset (for example, the code `8` should really be `008.0`) or the Single-level CCS crosswalk (`' and whitespace characters`). 

This is quite typical of healthcare data, unfortunately. Many of the publically available files are not suited for reading into modern programming languages. Often, they are limited to SAS format datasets, which is a proprietary software suite for working with statistical packages that is ubiquitous in health care.

In order to use the CCS groupings, we'll have to clean both the groupings *and* the diabetes data as well. Here is the general procedure that we will take (Although this is an imperfect mapping as well).

**Remove all quotes and extra whitespace from the codes and the column names in the CCS crosswalk [10 pts]**

In [18]:
dxref_df = dxref_df.rename(columns=lambda x: re.sub("\s+"," ",x))
dxref_df = dxref_df.rename(columns=lambda x: re.sub("'","",x))
dxref_df = dxref_df.apply(lambda x: x.str.strip("'")).apply(lambda x: x.str.strip(" "))


In [19]:
dxref_df.head()

,ICD-9-CM CODE,CCS CATEGORY,CCS CATEGORY DESCRIPTION,ICD-9-CM CODE DESCRIPTION,OPTIONAL CCS CATEGORY,OPTIONAL CCS CATEGORY DESCRIPTION
0,,0,No DX,INVALID CODES IN USER DATA,,
1,01000,1,Tuberculosis,PRIM TB COMPLEX-UNSPEC,,
2,01001,1,Tuberculosis,PRIM TB COMPLEX-NO EXAM,,
3,01002,1,Tuberculosis,PRIM TB COMPLEX-EXM UNKN,,
4,01003,1,Tuberculosis,PRIM TB COMPLEX-MICRO DX,,


Next, we're going to write a function that cleans up the ICD codes found in the Diabetes dataset. If you'll notice, there are no decimal points in the single-level CCS crosswalk. Therefore, we must make sure that our data matches that as well. Implement the function below, and we will use it in an `.apply()` call to modify the `diag_` columns.

#### Implement this function [15 pts]

In [20]:
def clean_diabetes_code(icd_code):
    """
    Formats codes found in the Diabetes dataset to be like those found in the CCS crosswalk
    
    
    If an icd_code has a decimal, remove the decimal
    
    if it has less than 3 digits, prepend '0's, until it is 3 digits.
    
    If it has 3 digits (before or after the above step), append a '0'
    
    Examples:
        250.13 -> 25013
        32 -> 0320
        315 -> 3150
        
    Args: 
        icd_code:
    
    Returns:
        formatted_code string: A formatted ICD-Code string according to the above criteria:
        
        
    """
    code_copy = str(icd_code) # Convert to string due to weird .apply behavior in Series
    if code_copy == 'nan':
        return np.nan  
    ### Your Code here: -- modify code_copy until it returns the reformatted code. Store this in formatted_code and return it
    else:
        if code_copy.find('.'):
            code_copy = code_copy.replace('.','')
        if len(code_copy)<3:
            code_copy = '0'* (3 - len(code_copy)) + code_copy
        if len(code_copy)==3:
            code_copy += '0'
        return code_copy

Now, replace all three `diag_` columns by calling .apply with this function. For example:

`diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(clean_diabetes_code)`

In [21]:
diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(clean_diabetes_code)
diabetes_df['diag_2'] = diabetes_df['diag_2'].apply(clean_diabetes_code)
diabetes_df['diag_3'] = diabetes_df['diag_3'].apply(clean_diabetes_code)
diabetes_df['diag_1']

0         25083
1          2760
2          6480
3          0080
4          1970
5          4140
6          4140
7          4280
8          3980
9          4340
10         2507
11         1570
12         4280
13         4280
14         5180
15         9990
16         4100
17         6820
18         4020
19         7370
20         4100
21         5720
22         4100
23         V570
24         1890
25         7860
26         4270
27         9960
28         2770
29         4280
          ...  
101736     5900
101737     1820
101738     2750
101739     4140
101740     4140
101741     8200
101742     2508
101743     4860
101744     4270
101745     6820
101746     4270
101747     4100
101748     2950
101749     7150
101750     5740
101751     4240
101752     3480
101753     2508
101754     5740
101755     5920
101756     9960
101757     4910
101758     2920
101759     4350
101760     3450
101761    25013
101762     5600
101763     0380
101764     9960
101765     5300
Name: diag_1, Length: 10

#### Join in the single-level CCS crosswalk and answer the following question: [15 pts]

List the top 10 condition categories (`CCS CATEGORY DESCRIPTION`) when you add up all instances over all 3 columns

In [22]:
diabetes_df =  pd.merge(diabetes_df,dxref_df[['ICD-9-CM CODE','CCS CATEGORY DESCRIPTION']],left_on = 'diag_1',
                       right_on = 'ICD-9-CM CODE', how = 'left')
diabetes_df =  pd.merge(diabetes_df,dxref_df[['ICD-9-CM CODE','CCS CATEGORY DESCRIPTION']],left_on = 'diag_2',
                       right_on = 'ICD-9-CM CODE', how = 'left')
diabetes_df =  pd.merge(diabetes_df,dxref_df[['ICD-9-CM CODE','CCS CATEGORY DESCRIPTION']],left_on = 'diag_3',
                       right_on = 'ICD-9-CM CODE', how = 'left')


In [23]:
diabetes_df = diabetes_df['CCS CATEGORY DESCRIPTION_x'].append(diabetes_df['CCS CATEGORY DESCRIPTION_y'])
diabetes_df.value_counts()[:10]

chf;nonhp       13524
Coron athero    13210
Fluid/elc dx     8641
Dysrhythmia      8419
Htn complicn     7883
DiabMel w/cm     7695
UTI              5045
Acute MI         4163
COPD             4110
Skin infectn     3540
dtype: int64